In [71]:
from chalk.client import ChalkClient
from chalk.features import features, online, has_one, DataFrame
from typing import Optional


c = ChalkClient(branch="main2")

In [72]:
@features
class CardAccountFS:
    id: int

    some_card_feature: str


Uploading 1 feature class to branch server...
*	feature: card_account_fs.id
*	feature: card_account_fs.some_card_feature


In [73]:
@features
class FundingSource:
    id: int
    card_account_id: Optional[int]
    bank_account_id: Optional[int]
    funding_source_type: str # card or account

    some_feature: str

    card_account: Optional[CardAccountFS] = has_one(lambda: CardAccountFS.id == FundingSource.card_account_id)

Uploading 1 feature class to branch server...
*	feature: funding_source.bank_account_id
*	feature: funding_source.card_account
*	feature: funding_source.card_account_id
*	feature: funding_source.funding_source_type
*	feature: funding_source.id
*	feature: funding_source.some_feature


In [74]:
@online
def funding_source_type(
        funding_source_type: FundingSource.funding_source_type,
        card_account_feature: Optional[FundingSource.card_account.some_card_feature]
) -> FundingSource.some_feature:
    return card_account_feature

Uploading 1 resolver to branch server...
*	resolver: funding_source_type


In [75]:
@online
def get_card_accounts() -> DataFrame[CardAccountFS.id, CardAccountFS.some_card_feature]:
    return DataFrame([
        CardAccountFS(id=100, some_card_feature="Something")
    ])

Uploading 1 resolver to branch server...
*	resolver: get_card_accounts


In [76]:
@online
def get_funding_sources() -> DataFrame[FundingSource.id, FundingSource.funding_source_type, FundingSource.card_account_id, FundingSource.bank_account_id]:
    return DataFrame([
        FundingSource(id=1, funding_source_type="card", card_account_id=100, bank_account_id=None),
        FundingSource(id=2, funding_source_type="bank_account", card_account_id=None, bank_account_id=1000)
    ])

Uploading 1 resolver to branch server...
*	resolver: get_funding_sources


In [77]:
c.query(input={FundingSource.id: 1}, output=[FundingSource.id, FundingSource.funding_source_type, FundingSource.bank_account_id, FundingSource.card_account_id, FundingSource.some_feature], explain=True)



## Features
```
┌───────────────────────────────────┬───────────┐
│ Feature                           ┆ Value     │
╞═══════════════════════════════════╪═══════════╡
│ funding_source.id                 ┆ 1         │
│ funding_source.card_account_id    ┆ 100       │
│ funding_source.some_feature       ┆ Something │
│ funding_source.bank_account_id    ┆ null      │
│ funding_source.funding_source_ty… ┆ card      │
└───────────────────────────────────┴───────────┘
```